In [1]:
import sys
!{sys.executable} -m pip install openai

In [2]:
import sys
!{sys.executable} -m pip install python-dotenv

In [13]:
import sys
!{sys.executable} -m pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.8/982.8 kB 4.4 MB/s eta 0:00:00


In [15]:
import os
import openai
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file



In [33]:
def get_llm(model="gpt-4o"):
   llm = ChatOpenAI(model=model,
   temperature=0,
   max_tokens=None,
   timeout=None,
   max_retries=2)
   llm.bind(response_format={"type": "json_object"})
   return llm

In [35]:
get_llm().invoke("Explain what is Ai Agent in layman terms").content

'An AI agent, in simple terms, is like a smart computer program that can make decisions and perform tasks on its own. Imagine it as a virtual assistant that can understand what you want, think about the best way to help you, and then do it without needing you to guide it every step of the way. For example, an AI agent could be a chatbot that answers your questions online, a virtual assistant like Siri or Alexa that helps you with tasks on your phone, or even a program that plays games against you. It uses artificial intelligence to learn from its experiences and improve over time, just like how people learn from practice.'

In [41]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("user", "Translate the text {text} to {language} and just give me the translated text as output")
])
chain = prompt | get_llm()

In [43]:
chain.invoke({"text": "How are you", "language":"French"}).content

'Comment ça va ?'

In [44]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [57]:
review_prompt_template = ChatPromptTemplate.from_template(review_template)
print(review_prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [58]:
review_chain = review_prompt_template | get_llm()


In [64]:
review_chain.invoke({"text":customer_review}).content

'```json\n{\n  "gift": true,\n  "delivery_days": 2,\n  "price_value": [\n    "It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."\n  ]\n}\n```'